## Решил попрактиковаться в сборе и обработке данных

#### Сегодня спарсим auto.ru и пихнем это в DataFrame для дальнейшей обработки

#### Я решил парсить только плюс-минус конкретные данные: 
#### ВАЗ до 100к рублей

In [267]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import numpy as np
import unidecode

#### По данным фильтрам выдало 32 страницы, поэтому делаем цикл на 32 элемента, парсим данные и записываем нужные в список

In [441]:
model_list = []
price_list = []
year_list = []
km_list = []
link_list = []
for i in range (1,32):
    resp = requests.get('https://auto.ru/moskovskaya_oblast/cars/vaz/all/do-100000/?page='+str(i))
    resp.encoding = 'utf-8'
    bs = BeautifulSoup(resp.text)
    for j in bs.find_all(class_='ListingItem__main'):
        for i in j.find_all(class_ = 'Link ListingItemTitle__link'):
            model_list.append(unidecode.unidecode(i.text))
        for i in j.find_all(class_ = 'ListingItem__priceBlock'):
            price_list.append(unidecode.unidecode(i.text.replace('₽','')).replace(' ',''))
        for i in j.find_all(class_ = 'ListingItem__yearBlock'):
            year_list.append(unidecode.unidecode(i.text))
        for i in j.find_all(class_ = 'ListingItem__kmAgeAndActionsBlock'):
            km_list.append(unidecode.unidecode(i.text.replace('км','')).replace(' ',''))
        for i in j.find_all(class_ = 'ListingItem__kmAgeAndActionsBlock'):
            km_list.append(unidecode.unidecode(i.text.replace('км','')).replace(' ',''))
        for i in j.find_all(class_='ListingItemTechSummaryDesktop__column'):
            link_list.append(unidecode.unidecode(i.text))

### Один технические хар-ти лежат не в самом удобном формате, поэтому прийдется потратить немного времени что бы обработать их

In [487]:
litr = []
dop = []
for i in tech_list:
    if len(i) > 20:
        litr.append(i.split('/'))
    else:
        dop.append(i)

### Я не придумал, как достать адекватно достать данные формата 'механикажелтый', поэтому забил на них

In [505]:
litr_list = []
ls_list = []
for i in litr:
    litr_list.append(i[0].replace('l','').strip())
    ls_list.append(i[1].replace('l.s.','').strip())
    

### На данном этапе я заметил артефакты

#### Это кривые ссылки

#### Причем ссылки на фотки рабочие и такие фотки действительно есть на сайте. 

#### Но зачастую цены не соответсвуют

### Создаю ДатаФрейм и кладу полученные значения в него

In [507]:
columns = ['model', 'year', 'km', 'litr', 'hp', 'price']

In [593]:
df = pd.DataFrame(columns=columns)

In [594]:
df

,model,year,km,litr,hp,price


In [595]:
df.model = model_list

In [596]:
df.year = year_list

In [597]:
df.km = km_list[:1146]

In [598]:
df.price = price_list

In [599]:
df.hp = ls_list

In [600]:
df.litr = litr_list

In [601]:
df.head()

,model,year,km,litr,hp,price
0,LADA (VAZ) 2112,2003,227000,1.5,72,80000
1,LADA (VAZ) 2102,1985,227000,1.2,64,80000
2,LADA (VAZ) 2108,2001,102000,1.3,135,80000
3,LADA (VAZ) 2105,2000,102000,1.5,75,70000
4,LADA (VAZ) 2112,2004,280000,1.8,98,110000


### Все круто, кроме одного момента в данных некоторых колонок затесались артефакты вроде '166000ProverennyidilerAvtoGERMESAvtosprobegomIaroslavskoesh'

### Нужно от них избавиться чтобы привести данные к верному типу данных

In [619]:
df.dtypes

model            object
year     datetime64[ns]
km                int64
litr            float64
hp                int64
price             int64
dtype: object

### Объективно, если пробег у машины больше, чем миллион, то стоит задуматься над верностью данных, так что отсеиваем все данные где символов больше 7

In [609]:
for i in df.km:
    if len(i) > 7:
        df = df.loc[~df['km'].isin([f"{i}"])]

TypeError: object of type 'int' has no len()

### С ценой все еще проще, тк я указал максимальнйы прайс в 100 000, но поленился и оставил так же 7 символов

In [613]:
for i in df.price:
    if len(i) > 7:
        df = df.loc[~df['price'].isin([f"{i}"])]

### Ну все, приводим к нужному типу данных

In [607]:
df.km = df.km.astype(int)

In [610]:
df.litr = df.litr.astype(float)

In [611]:
df.hp = df.hp.astype(int)

In [614]:
df.price = df.price.astype(int)

In [615]:
df.year = pd.to_datetime(df.year, format='%Y')

In [616]:
df.head()

,model,year,km,litr,hp,price
0,LADA (VAZ) 2112,2003-01-01,227000,1.5,72,80000
1,LADA (VAZ) 2102,1985-01-01,227000,1.2,64,80000
2,LADA (VAZ) 2108,2001-01-01,102000,1.3,135,80000
3,LADA (VAZ) 2105,2000-01-01,102000,1.5,75,70000
4,LADA (VAZ) 2112,2004-01-01,280000,1.8,98,110000


In [618]:
df.shape

(1126, 6)

П